In [2]:
import numpy as np
import struct
import matplotlib.pyplot as plt

Границы зон затопления определяются двумя кусочно-линейными функциями. Область ниже кусочно-линейной функции определяемой наборами точек x_zon_1 и y_zon_1 затапливается из р. Ахтуба. Область выше кусочно-линейной функции определяемой наборами точек x_zon_2 и y_zon_2 затапливается из р. Волга. Область между двумя кусочно-линейными функциями затапливается из р. Ахтуба и р. Волга.

In [3]:
x_zon_1=[780, 540, 500, 402, 380, 300, 285, 284]
y_zon_1=[371, 370, 480, 505, 600, 636, 675, 940]
x_zon_2=[760, 500, 460, 362, 360, 240, 230, 229]
y_zon_2=[311, 320, 430, 455, 590, 626, 675, 940]

In [4]:
#Правый берег р. Волга
x_volga1=np.array([362,343,303,302,229,190,154,121,115,91,70,40,27,25])
x_volga2=np.array([26,33,46,75,81,109, 135,191,196,222,247,263,277,298,329,333,359,376,381,426,485,517,565,601,636,730,847,883,944])
y_volga1=np.array([940, 914,863,860,761,691,641,609,602,576,543,499,466,433])
y_volga2=np.array([424,390,361,323,320,295, 275,258,254,264,288,295,296,289,260,248,211,191,181,158,157,163,149,133,101,35,40,49,88])


In [5]:
def ReadFile(fname):
    """

    """
    try:
        f=open(fname, "rb")
        C=f.read(4).decode("utf-8")
        #print(C)
        N_x=int.from_bytes(f.read(2), "little") # кол-во ячеек по Х
        #print(N_x)
        N_y=int.from_bytes(f.read(2), "little") # кол-во ячеек по Y
        #print(N_y)
        P=struct.unpack('6d', f.read(6*8))
        #print(P)
        Matr = np.zeros ((N_x, N_y))
        # далее читаем матрицу построчно и записываем в Matr
        for i in range(N_x):
            for j in range(N_y):
                temp=struct.unpack('f', f.read(4))
                Matr[i][j]=temp[0]                      
    except IOError:
        print("An IOError has occurred!")
    finally:
        f.close()
    return Matr,P, N_x, N_y

def WriteFile(fname, id_surfer,N_x, N_y, AxisXY,  Matrix):
    """
    id_surfer (тип char) идентификатор (4-байтовая идентификационная строка 'DSBB', которая идентифицирует файл как файл сетки Surfer6binary)
    N_x (тип int) количество линий сетки по оси X (столбцов)
    N_y (тип int) количество линий сетки по оси Y (строк)
    AxisXY=[x_min, x_max, y_min, y_max, z_min, x_max], где  
    x_min (тип double) минимальное значение X сетки,
    x_max (тип double) максимальное значение X сетки,
    y_min (тип double) минимальное значение Y сетки,
    y_max (тип double) максимальное значение Y сетки,
    z_min (тип double) минимальное значение Z сетки,
    z_max (тип double) максимальное значение Z сетки,
    Matr
    """
    try:
        f=open(fname, "wb")
        f.write(id_surfer.encode("utf-8"))
        f.write(struct.pack('h', N_x))
        f.write(struct.pack('h', N_y))
        f.write(struct.pack('6d', AxisXY[0], AxisXY[1], AxisXY[2], AxisXY[3],  np.min(Matrix), np.max(Matrix)))
        for i in range(N_x):
            for j in range(N_y):
                f.write(struct.pack('f', Matrix[i][j]))        
    except IOError:
        print("An IOError has occurred!")
    finally:
        f.close()

In [6]:
def LineEquation (x_a, y_a, x_b, y_b, x):
    """
    Уравнение прямой проходящей через точки (x_a, y_a) и (x_b, y_b)
    """
    return ((y_b-y_a)/(x_b-x_a))*(x-x_a)+y_a
def PiecewiseLinearFunction (x_array, y_array, x):
    """
    С помощью уравнения прямой проходящей через 
    две точки А(x_array[i],y_array[i]) и B(x_array[i+1],y_array[i+1])
    задаем кусочно-линейную функцию для заданных наборов точек
    x_array, y_array
    """
    for i in range(len(x_array)-1):
        if x_array[i+1] <= x <= x_array[i]:
            return ((y_array[i+1]-y_array[i])/(x_array[i+1]-x_array[i]))*(x-x_array[i])+y_array[i]

In [7]:
def FloodingFromVolga(x_zon_2,  y_zon_2, x_0, y_0):
    """
    Проверка затапливается ли точка (x_0,y_0) из реки Волга.
    """
    
    if y_0 <= np.max(y_zon_2):
        return np.interp(x_0, x_zon_2[::-1], y_zon_2[::-1]) > y_0
def FloodingFromAkhtuba(x_zon_1,  y_zon_1, x_0, y_0):
    """
    Проверка затапливается ли точка (x_0,y_0) из реки Ахтуба.
    """
    if y_0 <= np.max(y_zon_1):
        return np.interp(x_0, x_zon_1[::-1], y_zon_1[::-1]) < y_0

In [8]:

x_volga=np.hstack([x_volga1, x_volga2])
y_volga=np.hstack([y_volga1, y_volga2])

def LineLength (x_array, y_array):
    """
    Вычисление длины ломанной-линии.
    """
    
    length=0
    for i in range(len(x_array)-1):
        length=length+50*np.linalg.norm(np.array([x_array[i+1],y_array[i+1]])-np.array([x_array[i],y_array[i]]))
        print (f'{i+1}, {x_array[i+1], y_array[i+1]}, {length}')
#LineLength (x_volga, y_volga )

def MinLength (length_zone, x_array, y_array):
    minLen=5000
    length_channel=0
    for i in range(len(x_array)-1):
        length_channel=length_channel+50*np.linalg.norm(np.array([x_array[i+1],y_array[i+1]])-np.array([x_array[i],y_array[i]]))
        if np.abs(length_zone-length_channel)<minLen:
            minLen=np.abs(length_zone-length_channel)
            point_number=i+1
    return point_number
#print(f'{MinLength (57000, x_array, y_array)}, {x_array[MinLength (57000, x_array, y_array)]}, {y_array[MinLength (57000, x_array, y_array)]}')

def PointNumber (x_zone, y_zone, x, y):
    for i in range(len(x_zone)-1):
        if (x_zone[i+1] <= x <= x_zone[i] and np.abs(y_zone[i] - y_zone[i+1])<15) or y_zone[i] <= y <= y_zone[i+1]:
            return i
            
        
def PointVolgaChannel(x_channel, y_channel, x_zone, y_zone, length_zone):
    """
     Вычисляются длины участков для склейки.
     Параметры??!!
    """
    x_len=[]
    y_len=[]
    for i in range(len(length_zone)-1):
        x_len=np.append(x_len, x_channel[MinLength (length_zone[i], x_channel, y_channel)])
        y_len=np.append(y_len, y_channel[MinLength (length_zone[i], x_channel, y_channel)])
    return x_len, y_len

def generator (Q, delta_Q, number_zones, day, N_x, N_y):
    """
    Для теста склейки
    """
    
    M = np.zeros ((N_x, N_y))
    for k in range(Q,Q+number_zones*delta_Q,delta_Q):
        for i in range(N_x):
            for j in range(N_y):
                M[i][j]=k
        fname="Q_"+str(int(k/10))+"_day_"+str(day)+".grd"
        WriteFile(fname, "DSBB", 944, 944, PP,   M)


def LoadingMapsVolga (Q, delta_Q, number_zones, day, path):
    N_x=944
    N_y=944
    H = np.zeros ((number_zones, N_x, N_y))
    for zone in range(number_zones):
        Q_str=str(int((Q+delta_Q*zone)/10))
        file_name=path+"расчет_2023_манинг_q"+Q_str+"_30day/H_    "+str(day)+".grd"
        H[zone][::][::],PP, N_x, N_y=ReadFile(file_name)
    return H


In [18]:
length_zone=np.array([5000,21000,37000,55000,88000])

x_length_volga, y_length_volga = PointVolgaChannel(x_volga, y_volga, x_zon_2, y_zon_2, length_zone)
x_zonVA=[230, 230, 360, 460]
y_zonVA=[860, 675, 590, 430]

print(x_length_volga)
print(y_length_volga)


Rel_test,PP, N_x, N_y=ReadFile("relief_base.grd")
Rel_test = np.zeros ((N_x, N_y))
Q=26250
delta_Q=-250
day=5
number_zones=5
path='Z:/Работа/Базовый корпус карт затопления/'


H_volga=LoadingMapsVolga (Q, delta_Q, number_zones, day, path)
file_name=path+"расчет_2023_манинг_q"+str(int(Q/10))+"_30day/H_    "+str(day)+".grd"
H_akhtuba,PP, N_x, N_y=ReadFile(file_name)


def GluingCards (x_zon_1,  y_zon_1, x_zon_2,  y_zon_2, x_length_volga, N_x, N_y, H_volga, H_volgaakhtuba, H_akhtuba):
    H_gluing = np.zeros ((N_x, N_y))
    for y in range(N_y):
        for x in range (N_x):
            if FloodingFromAkhtuba(x_zon_1,  y_zon_1, x, y):
                H_gluing[y][x]=H_akhtuba[y][x]
            elif FloodingFromVolga(x_zon_2,  y_zon_2, x, y):
                if LineEquation (x_length_volga[0], y_length_volga[0], x_zonVA[0], y_zonVA[0], x)<y:
                    H_gluing[y][x]=H_volga[0][y][x]
                if LineEquation (x_length_volga[len(x_length_volga)-1], y_length_volga[len(x_length_volga)-1], x_zonVA[len(x_length_volga)-1], y_zonVA[len(x_length_volga)-1], x)>y:
                    H_gluing[y][x]=H_volga[len(x_length_volga)][y][x]
                for point in range(1,len(x_length_volga)):
                    if LineEquation (x_length_volga[point], y_length_volga[point], x_zonVA[point], y_zonVA[point], x) <= y <= LineEquation (x_length_volga[point-1], y_length_volga[point-1], x_zonVA[point-1], y_zonVA[point-1], x):
                        H_gluing[y][x]=H_volga[point][y][x]              
            else:
                if LineEquation (x_length_volga[0], y_length_volga[0], x_zonVA[0], y_zonVA[0], x)<y:
                    H_gluing[y][x]=H_volga[0][y][x]
                if LineEquation (x_length_volga[len(x_length_volga)-1], y_length_volga[len(x_length_volga)-1], x_zonVA[len(x_length_volga)-1], y_zonVA[len(x_length_volga)-1], x)>y:
                    H_gluing[y][x]=H_volga[len(x_length_volga)][y][x]
                for point in range(1,len(x_length_volga)):
                    if LineEquation (x_length_volga[point], y_length_volga[point], x_zonVA[point], y_zonVA[point], x) <= y <= LineEquation (x_length_volga[point-1], y_length_volga[point-1], x_zonVA[point-1], y_zonVA[point-1], x):
                        H_gluing[y][x]=H_volga[point][y][x] 
    return H_gluing

            
            

Rel_test=GluingCards (x_zon_1,  y_zon_1, x_zon_2,  y_zon_2, x_length_volga, N_x, N_y, H_volga, H_volga, H_akhtuba)
write_path=path+"Склейка_карт/"+"!H_2 (1991г).grd"                   
WriteFile(write_path, "DSBB", 944,944, PP, Rel_test)

[302. 115.  75. 359.]
[860. 602. 323. 211.]
